In [ ]:
import param
import quest
import parambokeh
import holoviews as hv
import geoviews as gv
import xarray as xr
import cartopy.crs as ccrs

hv.extension('bokeh')

## Select tiles to download

In [ ]:
url_list = ['http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
            'https://s.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png',
            'http://tile.stamen.com/terrain/{Z}/{X}/{Y}.png',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/World_Physical_Map/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/USA_Topo_Maps/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{Z}/{Y}/{X}',
            'https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{Z}/{Y}/{X}']

class TileSelector(param.Parameterized):
    
    tile_url = param.ObjectSelector(default=url_list[0], objects=url_list)
    
    zoom_level = param.Integer(default=8, bounds=(0, 19))
    
    @param.parameterized.bothmethod
    def view(cls_or_self):
        tile = gv.WMTS(cls_or_self.tile_url).options(global_extent=True, width=500, height=500)
        return tile

parambokeh.Widgets(TileSelector)
boxes = gv.Polygons([]).options(fill_alpha=0.4, color='blue', line_color='blue', line_width=2)
box_stream = hv.streams.BoxEdit(source=boxes, num_objects=1)
TileSelector.view() * boxes

## Declare quest collection

In [ ]:
basemap_features = quest.api.get_features('svc://basemap:Web Mercator Tile Service')
collection_name = 'the_demo2'
if collection_name in quest.api.get_collections():
    pass
else:
    quest.api.new_collection(collection_name)

collection_feature = quest.api.add_features(collection_name, basemap_features[0])
collection_feature

## Define query and download

In [ ]:
if box_stream.element:
    data = box_stream.data
    bbox = (data['x0'][0], data['y1'][0], data['x1'][0], data['y0'][0])
else:
    bbox = [-90.88, 32.36, -90.87, 32.35]

options = {'url': TileSelector.tile_url, 
           'zoom_level': TileSelector.zoom_level, 
           'bbox': bbox}
staged_id = quest.api.stage_for_download(collection_feature, options=options)
quest.api.download_datasets(staged_id)

## Open and display downloaded tiles

In [ ]:
meta = quest.api.get_metadata(staged_id)
path = list(meta.values())[0]['file_path']
gv.tile_sources.Wikipedia * gv.RGB.load_tiff(path, crs=ccrs.PlateCarree()).options(width=800, height=600)